# Checks the exported data.

In [27]:
from projects.common import *
from projects.utils import *
from typing import Tuple
from tqdm import tqdm
# from tqdm.notebook import tqdm
from multiprocessing import Pool, RLock
from configobj import ConfigObj
import numpy as np
import getpass
import json
import math
import os
import psycopg2
import pandas as pd
import time

import matplotlib.pyplot as plt
%matplotlib inline


In [25]:
exported_files = [os.path.join(STRUCTURED_EXPORT_DIR, i)
                  for i in os.listdir(STRUCTURED_EXPORT_DIR)]


def func(files, pid):
    uids = []
    for f in tqdm(files):
        uids += load_dsv(f)['uid'].tolist()
    return uids


uid_list = parallel_processing(func, MP_NUM_PROCESSES, exported_files)


 49%|████▉     | 2640/5387 [00:26<00:30, 88.67it/s]/tmp/ipykernel_3434/1934198347.py:8: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.
  uids += load_dsv(f)['uid'].tolist()
100%|██████████| 5387/5387 [00:57<00:00, 93.25it/s] 


In [38]:
uid_values = [uid for uids in uid_list for uid in uids]
uid, count = np.unique(uid_values, return_counts=True)
df = pd.DataFrame({'uid': uid, 'occurence': count}).sort_values(by=['uid'])

V1_DATA_ITEM_PATH = "mimic-iv/projects/resources/d_v1.tsv"
df.to_csv('../../../'+V1_DATA_ITEM_PATH, na_rep='', sep='\t', index=False)


In [37]:
with open('../../../'+UID_INFO_PATH, 'r') as f:
    jd = json.load(f)

df = pd.DataFrame({'uid': list(jd.keys()), 'label': list(jd.values())})

V1_INFO_ITEM_PATH = "mimic-iv/projects/resources/i_v1.tsv"
df.to_csv('../../../'+V1_INFO_ITEM_PATH, na_rep='', sep='\t', index=False)
